## Data preprocess

In [56]:
from pathlib import Path
from openpyxl import Workbook # type: ignore
from pathlib import Path
import os


def to_excel(data, filename, sheet_style, result_dir_path):
    workbook = Workbook()
    sheet = workbook.active

    # | user-response-time | request-number | response-ip   | process-time  |
    # | 0.3                | 10000          | 192.168.0.150 | 14.523432     |
    # | 0.5                | 20000          | 192.168.0.151 | 9.5232642     |

    sheet.append(sheet_style)

    for row in data:
        sheet.append(row)

    

    if not os.path.exists(result_dir_path):
        os.makedirs(result_dir_path, exist_ok=True)

    workbook.save(filename=f"{result_dir_path}\\{filename}.xlsx")


# read input dataset
filename_prefix = (
    "RANDclientv_single_worker_node-L1-RB1000-DTWedSep41611532024"
)


dataset_read_filename = filename_prefix
training_data_dir = Path.cwd().parent / "training_data"
# Data preprocessing
file_path = f"{training_data_dir}\\{dataset_read_filename}.xlsx"


# set result output filename and path
result_suffix = "result"

result_dir_path = Path.cwd().parent / "results" / "result_requestNumber_processTime"
version_index = len([_ for _ in Path(result_dir_path).iterdir() if _.is_file()])
version = f"_v{version_index}"

result_name = "requestNumber#responseTime" + version

result_output_filename = f"{filename_prefix}{result_name}{result_suffix}"

In [57]:

import pandas as pd # type: ignore
import numpy as np # type: ignore


def read_data(filename):
    df = pd.read_excel(filename)

    columns = df.columns.to_list()

    data_dict = {col: df[col].to_list() for col in columns}

    return data_dict


def data_preprocess(filepath):
    data = read_data(filepath)
    # TODO more...

    # to numpy
    for key in data.keys():
        data[key] = np.array(data[key])
    
    return data

In [58]:
dataset = data_preprocess(file_path)

## DATA Style View

In [59]:
# # dataset is a dictionary

# for col_header, col_data_list in dataset.items():
#     print(col_header, col_data_list)

## XGBOOST - 1

In [60]:
# from sklearn.ensemble import RandomForestRegressor # type: ignore
# # Create XGBoost regression model
# model = RandomForestRegressor(
#     n_estimators=100,   # 基础学习器（树）的数量
#     max_depth=5,        # 树的最大深度
#     max_features=0.8,   # 随机选择特征的比例（与 colsample_bytree 类似）
#     min_samples_split=2,  # 每个节点分裂所需的最小样本数（可以用于调节过拟合）
#     min_samples_leaf=1,  # 每个叶子节点需要的最小样本数
#     random_state=42,     # 保持结果可复现
# )

In [61]:
# from sklearn.linear_model import SGDRegressor  # type: ignore

# # Create SGD Regressor model
# model = SGDRegressor(
#     max_iter=1000,     # 迭代次数
#     tol=1e-3,          # 收敛容忍度
#     alpha=0.01,        # 正则化项系数
#     random_state=42    # 保持结果可复现
# )


In [62]:
# from sklearn.linear_model import LinearRegression     # type: ignore


# # Create Linear Regression model
# model = LinearRegression()


In [63]:
# from sklearn.linear_model import Ridge       # type: ignore

# # Create Ridge Regression model
# model = Ridge(
#     alpha=0.1,        # 正则化强度
#     random_state=42    # 保持结果可复现
# )


In [64]:
# from sklearn.linear_model import ElasticNet         # type: ignore

# model = ElasticNet(alpha=0.1, l1_ratio=0.5)  # alpha 和 l1_ratio 控制正则化强度和比例

In [65]:
from sklearn.neighbors import KNeighborsRegressor           # type: ignore

model = KNeighborsRegressor(n_neighbors=5)  # n_neighbors 控制最近邻的数量


In [66]:
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.metrics import mean_squared_error # type: ignore

# # select value from dataset
X = np.array([dataset.get("request_num")]).T
y = dataset.get("processed_time")

# train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=35
)

# model fit
model.fit(X_train, y_train)

KNeighborsRegressor()

In [67]:
from sklearn.metrics import mean_absolute_error # type: ignore

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 0.001348872047502368
Mean Absolute Error: 0.024303255558013916
    real    |    pred   

    3.784573554992676     |     3.766119861602783      
    5.100839138031006     |     5.0396037101745605      
    0.8065564632415771     |     0.8272031307220459      
    0.08397722244262695     |     0.0714810848236084      
    2.898449897766113     |     2.918788480758667      
    0.09162354469299316     |     0.09224462509155273      
    4.110558271408081     |     4.145219564437866      
    4.19347095489502     |     4.1987227439880375      
    6.263468980789185     |     6.193533325195313      
    0.911158561706543     |     0.9093011379241943      
    8.254207611083984     |     8.250317573547363      
    7.902703285217285     |     7.957002353668213      
    5.927728652954102     |     5.9222437858581545      
    1.307764291763306     |     1.323023223876953      
    8.284693479537964     |     8.237194299697876      
    2.202848434448242     |     2.20139

In [68]:
Path.cwd()

WindowsPath('d:/model_fit/training/training_model')

In [69]:
import joblib      # type: ignore

model_dirname = Path.cwd() / 'modelsfile'
model_filename =  model_dirname / 'number_time_random_forest_model.joblib'
joblib.dump(model, model_filename)

['d:\\model_fit\\training\\training_model\\modelsfile\\number_time_random_forest_model.joblib']